In [3]:
import sys
import os
sys.path.append(os.getcwd() + "/..")
from ws.scraper import Scraper
from selenium import webdriver
from ws.constants import *
import json
import pandas as pd
import numpy as np

In [4]:
LEAGUES

{'Brasileirao': {'countryid': 31, 'leagueid': 95},
 'Bundesliga': {'countryid': 81, 'leagueid': 3},
 'Chinese Super League': {'countryid': 45, 'leagueid': 162},
 'Eredivisie': {'countryid': 155, 'leagueid': 13},
 'La Liga': {'countryid': 206, 'leagueid': 4},
 'Liga NOS': {'countryid': 177, 'leagueid': 21},
 'Ligue 1': {'countryid': 74, 'leagueid': 22},
 'Premier League': {'countryid': 252, 'leagueid': 2},
 'Primera Argentina': {'countryid': 11, 'leagueid': 68},
 'Russian Premier League': {'countryid': 182, 'leagueid': 77},
 'Serie A': {'countryid': 108, 'leagueid': 5},
 'Super Lig': {'countryid': 225, 'leagueid': 17},
 'UEFA Champions League': {'countryid': 250, 'leagueid': 12},
 'UEFA Europa League': {'countryid': 250, 'leagueid': 30}}

In [5]:
class League():
    
    jsonpath = JSONPATH
    
    def __init__(self,league):
        
        self.league = league
        self.leaguename = self.league.lower().replace(" ","_")
        self.leaguepath = self.jsonpath+self.leaguename+"/"
        self.seasons = list(filter(lambda x: "2" in x,os.listdir(self.leaguepath)))
        self.matcheslists = {s:list(map(lambda x: x.split(".")[0],list(filter(lambda x: ".json" in x,os.listdir(self.leaguepath+s))))) for s in self.seasons}
                
    def matchesList(self,season=None):
        
        return self.matcheslists[season] if season != None else self.matcheslists
    
    def numberMatches(self):
        
        return pd.Series({k:len(v) for k,v in self.matcheslists.items()},name="numberofmatches")
    
    

In [8]:
premier = League("Premier League")
premier.numberMatches()

2009-2010    380
2010-2011    380
2011-2012    380
2012-2013    380
2013-2014    380
2014-2015    380
2015-2016    380
2016-2017    380
2017-2018    300
Name: numberofmatches, dtype: int64

In [9]:
premier.matcheslists.keys()

dict_keys(['2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018'])

In [109]:
class Match():
    
    def __init__(self,matchid,league,season,jsonpath=JSONPATH):
        
        self.league = league
        self.season = season
        self.leaguename = self.league.lower().replace(" ","_")
        self.matchid = matchid
        self.jsonpath = jsonpath
        self.matchpath = jsonpath+self.leaguename+"/"+self.season+"/"+str(self.matchid)+".json"
        self.match = json.load(open(self.matchpath,"r",encoding="utf-8"),encoding="utf-8")
        
        #self.__matchInfo__()
        #self.__events__()
        #self.__players__()
        #self.__teams__()
        
    def __matchInfo__(self):
        
        matches_dict = {}
        
        try:
            matches_dict["wsmatchid"] = self.matchid
        except KeyError:
            matches_dict["wsmatchid"] = np.nan
        try:
            matches_dict["league"] = self.league
        except KeyError:
            matches_dict["league"] = np.nan
        try:
            matches_dict["season"] = self.season
        except KeyError:
            matches_dict["season"] = np.nan
        try:
            matches_dict["date"] = pd.to_datetime(self.match["startTime"].replace("T"," "))
        except KeyError:
            matches_dict["date"] = np.nan
        try:
            matches_dict["hometeamid"] = self.match["home"]["teamId"]
        except KeyError:
            matches_dict["hometeamid"] = np.nan
        try:
            matches_dict["awayteamid"] = self.match["away"]["teamId"]
        except KeyError:
            matches_dict["awayteamid"] = np.nan
        try:
            matches_dict["hometeamname"] = self.match["home"]["name"]
        except KeyError:
            matches_dict["hometeamname"] = np.nan
        try:
            matches_dict["awayteamname"] = self.match["away"]["name"]
        except KeyError:
            matches_dict["awayteamname"] = np.nan
        try:
            matches_dict["homescore"] = self.match["score"].split(" : ")[0]
        except KeyError:
            matches_dict["homescore"] = np.nan
        try:
            matches_dict["awayscore"] = self.match["score"].split(" : ")[1]
        except KeyError:
            matches_dict["awayscore"] = np.nan
        try:
            if self.match["pkScore"] == '':
                matches_dict["homepkscore"] = np.nan
                matches_dict["awaypkscore"] = np.nan
            else:
                matches_dict["homepkscore"] = self.match["pkScore"].split(" : ")[0]
                matches_dict["awaypkscore"] = self.match["pkScore"].split(" : ")[1]
        except KeyError:
            matches_dict["homepkscore"] = np.nan
            matches_dict["awaypkscore"] = np.nan
        try:
            matches_dict["referee"] = self.match["referee"]["name"]
        except KeyError:
            matches_dict["referee"] = np.nan
        try:
            matches_dict["managerhome"] = self.match["home"]["managerName"]
        except KeyError:
            matches_dict["managerhome"] = np.nan
        try:
            matches_dict["manageraway"] = self.match["away"]["managerName"]
        except KeyError:
            matches_dict["manageraway"] = np.nan
        try:
            matches_dict["attendance"] = self.match["attendance"]
        except KeyError:
            matches_dict["attendance"] = np.nan
        try:
            matches_dict["venuename"] = self.match["venueName"]
        except KeyError:
            matches_dict["venuename"] = np.nan
        
        column_order = ["wsmatchid","league","season","date","hometeamid",
                        "awayteamid","hometeamname","awayteamname","homescore",
                        "awayscore","homepkscore","awaypkscore","referee",
                        "managerhome","manageraway","attendance","venuename"]
        
        self.matchinfo = pd.Series(matches_dict)[column_order]
        
        return self.matchinfo
    
    def __events__(self):
            
        events = self.match["events"]
        event_list=[]
        qual_list=[]

        for event in events:
            events_dict = {}
            try:
                events_dict["keyid"] = str(self.matchid)+str(event["id"])
            except KeyError:
                events_dict["keyid"] = np.nan
            try:
                events_dict["wsmatchid"] = self.matchid
            except KeyError:
                events_dict["wsmatchid"] = np.nan
            try:
                events_dict["wseventid"] = str(event["id"])
            except KeyError:
                events_dict["wseventid"] = np.nan
            try:
                events_dict["matcheventid"] = event["eventId"]
            except KeyError:
                events_dict["matcheventid"] = np.nan
            try:
                events_dict["minute"] = event["minute"]
            except KeyError:
                events_dict["minute"] = np.nan
            try:
                events_dict["second"] = event["second"]
            except KeyError:
                events_dict["second"] = np.nan
            try:
                events_dict["expandedminute"] = event["expandedMinute"]
            except KeyError:
                events_dict["expandedminute"] = np.nan
            try:
                events_dict["teamid"] = event["teamId"]
            except KeyError:
                events_dict["teamid"] = np.nan
            try:
                events_dict["playerid"] = event["playerId"]
            except KeyError:
                events_dict["playerid"] = np.nan
            try:
                events_dict["period"] = event["period"]["displayName"]
            except KeyError:
                events_dict["period"] = np.nan
            try:
                events_dict["typeid"] = event["type"]["value"]
            except KeyError:
                events_dict["typeid"] = np.nan
            try:
                events_dict["type"] = event["type"]["displayName"]
            except KeyError:
                events_dict["type"] = np.nan
            try:
                events_dict["outcometype"] = event["outcomeType"]["displayName"]
            except KeyError:
                events_dict["outcometype"] = np.nan
            try:
                events_dict["x"] = event["x"]
            except KeyError:
                events_dict["x"] = np.nan
            try:
                events_dict["y"] = event["y"]
            except KeyError:
                events_dict["y"] = np.nan

            event_list.append(events_dict)

            qualifiers = event["qualifiers"]
            
            for qualifier in qualifiers:
                    qual_dict = {}
                    try:
                        qual_dict["keyid"] = events_dict["keyid"]
                    except KeyError:
                        qual_dict["keyid"] = np.nan
                    try:
                        qual_dict["wsmatchid"] = self.matchid
                    except KeyError:
                        qual_dict["wsmatchid"] = np.nan
                    try:
                        qual_dict["wseventid"] = events_dict["wseventid"]
                    except KeyError:
                        qual_dict["wseventid"] = np.nan
                    try:
                        qual_dict["matcheventid"] = events_dict["matcheventid"]
                    except KeyError:
                        qual_dict["matcheventid"] = np.nan
                    try:
                        qual_dict["qualid"] = qualifier["type"]["value"]
                    except KeyError:
                        qual_dict["qualid"] = np.nan
                    try:
                        qual_dict["qualname"] = qualifier["type"]["displayName"]
                    except KeyError:
                        qual_dict["qualname"] = np.nan
                    try:
                        try:
                            value = float(qualifier["value"])
                        except ValueError:
                            value = qualifier["value"]
                        if type(value) == str:
                            qual_dict["qualvaluetext"] = value
                            qual_dict["qualvalue"] = np.nan
                        else:
                            qual_dict["qualvaluetext"] = np.nan
                            qual_dict["qualvalue"] = value    
                    except KeyError:
                        qual_dict["qualvalue"] = 1
                        qual_dict["qualvaluetext"] = np.nan
                    qual_list.append(qual_dict)
    
        self.events = pd.DataFrame(event_list)[["keyid","wsmatchid","wseventid","matcheventid","minute","second","expandedminute","teamid",
                                   "playerid","period","typeid","type","outcometype","x","y"]].set_index("keyid")
        
        self.qualifiers = pd.DataFrame(qual_list)[["keyid","wsmatchid","wseventid","matcheventid","qualid","qualname","qualvalue","qualvaluetext"]].set_index("keyid")
        
        return self.events, self.qualifiers
        
        
        
    def __players__(self):
        
        dic = [{"playerid":k,"playername":v} for k,v in self.match["playerIdNameDictionary"].items()]
        self.players = pd.DataFrame(dic,index=range(len(dic))).set_index("playerid")
        
        return self.players
        
    def __teams__(self):
        
        d = {"home":{"teamid":self.matchinfo["hometeamid"],
                "teamname":self.matchinfo["hometeamname"]},
         
        "away":{"teamid":self.matchinfo["awayteamid"],
                "teamname":self.matchinfo["awayteamname"]}}
        
        self.teams = pd.DataFrame(d).T
        
        return self.teams
        

In [83]:
%%time
data = {}

for k,v in a.matcheslists.items():

    for match in v:

        try:
            m = ImportMatch(a.league,k,match)
            data["%s - %s vs %s" % (k,m.teams.loc["home"]["teamname"],m.teams.loc["away"]["teamname"])] = m
        except:
            print("ERROR",match)
    print(k,"finished")


2009-2010 finished
2010-2011 finished
2011-2012 finished
2012-2013 finished
2013-2014 finished
2014-2015 finished
2015-2016 finished
2016-2017 finished
2017-2018 finished
CPU times: user 6min 9s, sys: 4min 51s, total: 11min
Wall time: 11min 57s


In [54]:
m = data["Chelsea vs Burnley"]

att = m.events.groupby(["playerid","type"]).size().reset_index().set_index("playerid").merge(m.players,right_index=True,left_index=True)

In [84]:
events_all = pd.concat([v.events for k,v in data.items()])

In [85]:
events_all.shape

(5190552, 14)

In [89]:
import pickle
with open(LISTPATH+"events_list.pkl","wb") as f:
    pickle.dump(list(pd.unique(events_all["type"])),f)

In [91]:
qualifiers_all = pd.concat([v.qualifiers.qualname for k,v in data.items()])

In [92]:
qualifiers_all.shape

(21539552,)

In [109]:
matches_all = pd.concat([v.matchinfo for k,v in data.items()],axis=1).T.set_index("wsmatchid")

,wsmatchid,league,season,date,hometeamid,awayteamid,hometeamname,awayteamname,homescore,awayscore,homepkscore,awaypkscore,referee,managerhome,manageraway,attendance,venuename
0,317783,Premier League,2009-2010,2009-08-16 13:30:00,32,157,Man Utd,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
1,317786,Premier League,2009-2010,2009-08-16 16:00:00,30,26,Tottenham,Liverpool,2,1,NaN,NaN,Phil Dowd,Harry Redknapp,Rafael Benitez,35935,White Hart Lane
2,317787,Premier League,2009-2010,2009-08-15 15:00:00,24,194,Aston Villa,Wigan,0,2,NaN,NaN,Mark Clattenburg,Martin O'Neill,Roberto Martinez,35578,Villa Park
3,317788,Premier League,2009-2010,2009-08-15 17:30:00,31,13,Everton,Arsenal,1,6,NaN,NaN,Mark Halsey,David Moyes,Arsene Wenger,39309,Goodison Park
4,317789,Premier League,2009-2010,2009-08-15 15:00:00,161,29,Wolves,West Ham,0,2,NaN,NaN,Chris Foy,Mick McCarthy,Gianfranco Zola,28674,Molineux
5,317790,Premier League,2009-2010,2009-08-15 15:00:00,158,167,Blackburn,Man City,0,2,NaN,NaN,Mike Dean,Sam Allardyce,Mark Hughes,29584,Ewood Park
6,317791,Premier League,2009-2010,2009-08-15 15:00:00,92,16,Bolton,Sunderland,0,1,NaN,NaN,Andre Marriner,Gary Megson,Steve Bruce,22247,Reebok Stadium
7,317792,Premier League,2009-2010,2009-08-15 15:00:00,169,170,Portsmouth,Fulham,0,1,NaN,NaN,Martin Atkinson,Paul Hart,Roy Hodgson,17510,Fratton Park
8,317793,Premier League,2009-2010,2009-08-15 15:00:00,96,184,Stoke,Burnley,2,0,NaN,NaN,Steve Bennett,Tony Pulis,Owen Coyle,27385,Britannia Stadium
9,317794,Premier League,2009-2010,2009-08-15 12:45:00,15,214,Chelsea,Hull,2,1,NaN,NaN,Alan Wiley,Carlo Ancelotti,Phil Brown,41597,Stamford Bridge


In [95]:
with open(LISTPATH+"qualifiers_list.pkl","wb") as f:
    pickle.dump(list(pd.unique(qualifiers_all)),f)

In [112]:
events_all[events_all["type"] == "Clearance"].merge(matches_all,right_index=True,left_on=["wsmatchid"]).set_index("date").groupby

,wsmatchid,wseventid,matcheventid,minute,second,expandedminute,teamid,playerid,period,typeid,...,awayteamname,homescore,awayscore,homepkscore,awaypkscore,referee,managerhome,manageraway,attendance,venuename
date,,,,,,,,,,,,,,,,,,,,,
2009-08-16 13:30:00,317783,1111421993.0,12.0,1,25.0,1,157,1354.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,2055343342.0,33.0,2,33.0,2,32,22079.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,863166870.0,36.0,3,38.0,3,157,1344.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,861013874.0,38.0,3,50.0,3,157,4737.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,1012333097.0,39.0,3,52.0,3,157,29086.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,10293485.0,62.0,5,57.0,5,157,1344.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,852358479.0,117.0,8,14.0,8,32,44031.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,1102060065.0,119.0,8,21.0,8,32,3841.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford
2009-08-16 13:30:00,317783,861486890.0,121.0,8,29.0,8,32,22079.0,FirstHalf,12,...,Birmingham,1,0,NaN,NaN,Lee Mason,Alex Ferguson,Alex McLeish,75062,Old Trafford


In [56]:
todo = m.events.groupby(["playerid","type","outcometype"]).size().reset_index().set_index("playerid").merge(m.players,right_index=True,left_index=True)

In [62]:
suc = todo[todo["outcometype"] == "Successful"]

In [67]:
print(att.shape)
print(suc.shape)
print(todo.shape)

(257, 3)
(219, 4)
(315, 4)


In [72]:
att

,type,0,playername
playerid,,,
3860.0,Aerial,2,Jonathan Walters
3860.0,BallTouch,1,Jonathan Walters
3860.0,Dispossessed,2,Jonathan Walters
3860.0,Pass,3,Jonathan Walters
3860.0,SubstitutionOn,1,Jonathan Walters
8040.0,Aerial,1,Cesc Fàbregas
8040.0,BallRecovery,6,Cesc Fàbregas
8040.0,BallTouch,2,Cesc Fàbregas
8040.0,BlockedPass,1,Cesc Fàbregas


In [1]:
from dask import bag

In [33]:
files = list(filter(lambda x: ".json" in x,os.listdir(premier.leaguepath+"2017-2018")))

In [34]:
b = bag.from_sequence(premier.matcheslists["2017-2018"])

In [35]:
lista = b.map(lambda x: Match(x,"Premier League","2017-2018")).compute()

Process ForkPoolWorker-40:
KeyboardInterrupt
Process ForkPoolWorker-38:
Traceback (most recent call last):
Traceback (most recent call last):
Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x10ef42950>
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cgnal/anaconda/lib/python3.6/_weakrefset.py", line 38, in _remove
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
    def _remove(item, selfref=ref(self)):
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/queues.py", line 342, in get
    res = self._reader.recv_bytes()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_b

In [29]:
lista = b.compute()

In [30]:
lista

['1190174',
 '1190175',
 '1190176',
 '1190177',
 '1190178',
 '1190179',
 '1190180',
 '1190181',
 '1190182',
 '1190183',
 '1190184',
 '1190185',
 '1190186',
 '1190187',
 '1190188',
 '1190189',
 '1190190',
 '1190191',
 '1190192',
 '1190193',
 '1190194',
 '1190195',
 '1190196',
 '1190197',
 '1190198',
 '1190199',
 '1190200',
 '1190201',
 '1190202',
 '1190203',
 '1190204',
 '1190205',
 '1190206',
 '1190207',
 '1190208',
 '1190209',
 '1190210',
 '1190211',
 '1190213',
 '1190214',
 '1190215',
 '1190216',
 '1190217',
 '1190218',
 '1190219',
 '1190220',
 '1190221',
 '1190222',
 '1190223',
 '1190224',
 '1190225',
 '1190226',
 '1190227',
 '1190228',
 '1190229',
 '1190230',
 '1190231',
 '1190232',
 '1190233',
 '1190234',
 '1190235',
 '1190236',
 '1190237',
 '1190238',
 '1190239',
 '1190240',
 '1190241',
 '1190242',
 '1190243',
 '1190244',
 '1190245',
 '1190246',
 '1190247',
 '1190248',
 '1190249',
 '1190250',
 '1190251',
 '1190252',
 '1190253',
 '1190254',
 '1190255',
 '1190256',
 '1190257',
 '11

In [37]:
data = {}
for file in premier.matcheslists["2017-2018"]:
    
    data[file] = Match(file,"Premier League","2017-2018")

KeyboardInterrupt: 

In [40]:
data.keys()

dict_keys(['1190174', '1190175', '1190176', '1190177', '1190178', '1190179', '1190180', '1190181', '1190182', '1190183', '1190184', '1190185', '1190186', '1190187', '1190188', '1190189', '1190190', '1190191', '1190192', '1190193', '1190194', '1190195', '1190196', '1190197', '1190198', '1190199', '1190200', '1190201', '1190202', '1190203', '1190204', '1190205', '1190206', '1190207', '1190208', '1190209', '1190210', '1190211', '1190213', '1190214', '1190215', '1190216', '1190217', '1190218', '1190219', '1190220', '1190221', '1190222', '1190223', '1190224', '1190225', '1190226', '1190227', '1190228', '1190229', '1190230', '1190231', '1190232', '1190233', '1190234', '1190235', '1190236', '1190237', '1190238', '1190239', '1190240', '1190241', '1190242', '1190243', '1190244', '1190245', '1190246', '1190247', '1190248', '1190249', '1190250', '1190251', '1190252', '1190253', '1190254', '1190255', '1190256', '1190257', '1190258', '1190259', '1190260', '1190261', '1190262', '1190263', '1190264',

In [42]:
data["1190174"].teams

AttributeError: 'Match' object has no attribute 'teams'

In [95]:
%%time
a = bag.read_text(JSONPATH+'premier_league/2017-2018/*.json').map(json.loads).compute()

Process ForkPoolWorker-118:
KeyboardInterrupt
Process ForkPoolWorker-117:
Process ForkPoolWorker-119:
Process ForkPoolWorker-115:
Process ForkPoolWorker-121:
Process ForkPoolWorker-120:
Process ForkPoolWorker-116:
Process ForkPoolWorker-122:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/cgnal/anaconda/lib/python3.6/mult

In [84]:
b = a.compute

In [85]:
b

<bound method DaskMethodsMixin.compute of dask.bag<map-Mat..., npartitions=300>>

In [80]:
import dask
c = a.to_delayed()


In [ ]:
for i in 

In [71]:
b = Match(premier.matcheslists["2017-2018"][0],"Premier League","2017-2018")

In [72]:
b.match["home"].keys()

dict_keys(['teamId', 'formations', 'stats', 'incidentEvents', 'shotZones', 'name', 'countryName', 'players', 'managerName', 'scores', 'field', 'averageAge'])

In [73]:
%%time
lista = []
for s in files:
    with open(JSONPATH+'premier_league/2017-2018/'+s,"r") as f:
        lista.append(json.load(f)["home"]["teamId"])
    


CPU times: user 14 s, sys: 218 ms, total: 14.2 s
Wall time: 14.3 s


In [74]:
lista == a

True

In [59]:
files

['1190174.json',
 '1190175.json',
 '1190176.json',
 '1190177.json',
 '1190178.json',
 '1190179.json',
 '1190180.json',
 '1190181.json',
 '1190182.json',
 '1190183.json',
 '1190184.json',
 '1190185.json',
 '1190186.json',
 '1190187.json',
 '1190188.json',
 '1190189.json',
 '1190190.json',
 '1190191.json',
 '1190192.json',
 '1190193.json',
 '1190194.json',
 '1190195.json',
 '1190196.json',
 '1190197.json',
 '1190198.json',
 '1190199.json',
 '1190200.json',
 '1190201.json',
 '1190202.json',
 '1190203.json',
 '1190204.json',
 '1190205.json',
 '1190206.json',
 '1190207.json',
 '1190208.json',
 '1190209.json',
 '1190210.json',
 '1190211.json',
 '1190213.json',
 '1190214.json',
 '1190215.json',
 '1190216.json',
 '1190217.json',
 '1190218.json',
 '1190219.json',
 '1190220.json',
 '1190221.json',
 '1190222.json',
 '1190223.json',
 '1190224.json',
 '1190225.json',
 '1190226.json',
 '1190227.json',
 '1190228.json',
 '1190229.json',
 '1190230.json',
 '1190231.json',
 '1190232.json',
 '1190233.json

In [125]:
jobs = []


for match in premier.matcheslists["2017-2018"]:
        
        a = dask.delayed(Match)(match,"Premier League","2017-2018")
        b = dask.delayed(lambda x: x.__events__()[0])(a)
        jobs.append(b)
    

In [133]:
%%time
m = []
for match in premier.matcheslists["2017-2018"]:
    
    m.append(Match(match,"Premier League","2017-2018").__teams__())

AttributeError: 'Match' object has no attribute 'matchinfo'

In [128]:
jobs

[Delayed('lambda-0e4316b6-e0ae-4a84-937c-eebf47568260'),
 Delayed('lambda-27ef428d-0539-4b8f-8a1e-9a655cf44198'),
 Delayed('lambda-8544da49-6f24-4b12-92a7-0d652f40b429'),
 Delayed('lambda-f8949f6a-fbb1-465a-8441-674dfeabcba5'),
 Delayed('lambda-e5bf3c2a-c100-45a5-9072-4bab4ad855dc'),
 Delayed('lambda-1481267f-7d20-4391-876e-93e50f6a55c7'),
 Delayed('lambda-b35670cc-7439-4867-93c7-2c49dd7cb66c'),
 Delayed('lambda-e4f3a94e-47e9-4efb-bf35-c45afa3a5253'),
 Delayed('lambda-4ad5e5fe-8780-4080-9c16-44ecc7a4bc78'),
 Delayed('lambda-fc445b69-5ad0-4652-98b5-787b38d78b54'),
 Delayed('lambda-a0eeab6a-811c-4870-982a-f664b5f0b76b'),
 Delayed('lambda-50e83ade-fb36-4a51-8872-f97711390aee'),
 Delayed('lambda-f42e1a05-4c7f-47a3-9095-84cc7f67fe82'),
 Delayed('lambda-09840f95-4b14-4b71-8ad5-cfe10e5f7137'),
 Delayed('lambda-a5825d03-7150-429c-b062-b0801232fb53'),
 Delayed('lambda-91e3a5e7-e76c-48c6-9fac-c179241586e1'),
 Delayed('lambda-de196541-3ead-4883-85df-149946312a88'),
 Delayed('lambda-bb418809-1a71-

In [123]:
%%time
aaa = dask.delayed(lambda x: pd.concat([i.__events__()[0] for i in x]))(jobs)


CPU times: user 788 µs, sys: 282 µs, total: 1.07 ms
Wall time: 2.93 ms


In [129]:
%%time
aaa = dask.compute(jobs)

CPU times: user 32.3 s, sys: 1.39 s, total: 33.7 s
Wall time: 32.8 s


In [131]:
pd.concat(aaa[0])

,wsmatchid,wseventid,matcheventid,minute,second,expandedminute,teamid,playerid,period,typeid,type,outcometype,x,y
keyid,,,,,,,,,,,,,,
11901741985789402.0,1190174,1985789402.0,2.0,0,0.0,0,14,NaN,FirstHalf,32,Start,Successful,0.0,0.0
11901741969703302.0,1190174,1969703302.0,3.0,0,0.0,0,13,NaN,FirstHalf,32,Start,Successful,0.0,0.0
11901742073173630.0,1190174,2073173630.0,4.0,0,1.0,0,13,73078.0,FirstHalf,1,Pass,Successful,50.0,50.7
11901741757171580.0,1190174,1757171580.0,5.0,0,2.0,0,13,288795.0,FirstHalf,1,Pass,Successful,29.7,26.7
11901741222799094.0,1190174,1222799094.0,6.0,0,5.0,0,13,13756.0,FirstHalf,1,Pass,Successful,52.8,21.3
11901741753357194.0,1190174,1753357194.0,7.0,0,7.0,0,13,125209.0,FirstHalf,1,Pass,Successful,44.0,19.6
11901741534782070.0,1190174,1534782070.0,8.0,0,9.0,0,13,125211.0,FirstHalf,1,Pass,Successful,51.0,4.2
11901741540985821.0,1190174,1540985821.0,9.0,0,11.0,0,13,13756.0,FirstHalf,1,Pass,Unsuccessful,70.8,5.5
11901741149035280.0,1190174,1149035280.0,3.0,0,13.0,0,14,99487.0,FirstHalf,1,Pass,Unsuccessful,25.6,84.1


In [104]:
a = aaa[0][0].__players__()

In [105]:
a.players

AttributeError: 'NoneType' object has no attribute 'players'